<a href="https://colab.research.google.com/github/AbidYousuf/Abid-Yousuf/blob/main/ABoostAlgo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Load the dataset
data = pd.DataFrame({
    'Height': [5.5, 5.7, 5.4, 5.0, 5.8, 6.0, 5.9, 6.1, 5.6, 6.2],
    'Weight': [72, 75, 70, 60, 70, 71, 76, 70, 70, 75],
    'Eye color': ['Brown', 'Black', 'Brown', 'Black', 'Brown', 'Black', 'Black', 'Brown', 'Brown', 'Black'],
    'Gender': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'F', 'M']
})

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score


In [ ]:
# Define the dataset
X = np.array([
    [5.5, 72, 1, 0],
    [5.7, 75, 0, 1],
    [5.4, 70, 1, 1],
    [5.0, 60, 0, 0],
    [5.8, 70, 1, 0],
    [6.0, 71, 0, 1],
    [5.9, 76, 0, 1],
    [6.1, 70, 1, 1],
    [5.6, 70, 1, 0],
    [6.2, 75, 0, 1]
])

y = np.array([0, 1, 1, 0, 0, 1, 1, 1, 0, 1])


In [ ]:
# Initialize weights
n_samples = len(X)
weights = np.full(n_samples, 1 / n_samples)

# Initialize weak learner (decision stump)
def decision_stump(X, y, weights):
    n_samples, n_features = X.shape
    best_error = float('inf')
    best_feature = None
    best_threshold = None
    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            p = 1
            misclassified = (X[:, feature] < threshold) != y
            error = np.dot(weights, misclassified)
            if error < best_error:
                best_error = error
                best_feature = feature
                best_threshold = threshold
    return best_feature, best_threshold, best_error



In [ ]:
def adaboost_train(X, y, n_estimators):
    n_samples = len(X)
    weights = np.full(n_samples, 1 / n_samples)
    models = []
    alphas = []
    for _ in range(n_estimators):
        feature, threshold, error = decision_stump(X, y, weights)
        alpha = 0.5 * np.log((1 - error) / max(error, 1e-16))
        predictions = np.where(X[:, feature] < threshold, 0, 1)
        weights *= np.exp(-alpha * y * predictions)
        weights /= np.sum(weights)
        models.append((feature, threshold))
        alphas.append(alpha)
    return models, alphas

n_estimators = 5
models, alphas = adaboost_train(X, y, n_estimators)


In [ ]:
X_test = np.array([
    [5.3, 70, 0, 0],
    [6.3, 78, 1, 1],
    [5.9, 68, 1, 0],
    [5.6, 65, 0, 0]
])
y_test = np.array([0, 1, 1, 0])


In [ ]:
def adaboost_predict(X, models, alphas):
    n_samples = len(X)
    n_estimators = len(models)
    predictions = np.zeros(n_samples)
    for t in range(n_estimators):
        feature, threshold = models[t]
        alpha = alphas[t]
        prediction = np.where(X[:, feature] < threshold, 0, 1)
        predictions += alpha * prediction
    predictions = np.where(predictions < 0.5, 0, 1)
    return predictions

# Predict on the test dataset
y_pred = adaboost_predict(X_test, models, alphas)


# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0
